In [1]:
!pip install -q timm pydicom

In [2]:
import os
import gc
import sys
from PIL import Image
import cv2
import math, random
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

from collections import OrderedDict

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW

import timm
from timm.utils import ModelEmaV2
from transformers import get_cosine_schedule_with_warmup

import albumentations as A

from sklearn.model_selection import KFold

import re
import pydicom

In [3]:
INPUT_PATH  = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification'
# INPUT_PATH  = '/content/drive/My Drive/Kaggle/rsna-2024-lumbar-spine-degenerative-classification'
# INPUT_PATH  = '/Users/star-bits/Downloads/rsna-2024-lumbar-spine-degenerative-classification'
OUTPUT_PATH = '/kaggle/working'
# OUTPUT_PATH = '/content/drive/My Drive/Kaggle/rsna-2024-lsdc-train'
# OUTPUT_PATH = '/Users/star-bits/Downloads/rsna-2024-lsdc-train'

In [4]:
# Add Input -> Your Work + Notebook -> rsna24lsdc_0816_train
MODEL_PATH = '/kaggle/input/rsna24lsdc-0816-train/rsna-2024-lsdc-trained-models'

In [19]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
N_WORKERS = os.cpu_count()
USE_AMP = True
SEED = 8620

IMG_SIZE = [512, 512]
IN_CHANS = 40
N_LABELS = 25
N_CLASSES = 3 * N_LABELS

N_FOLDS = 5

MODEL_NAME = "tf_efficientnet_b3.ns_jft_in1k"

BATCH_SIZE = 1

In [20]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda', index=0)

In [21]:
df = pd.read_csv(f'{INPUT_PATH}/test_series_descriptions.csv')
df.head()

,study_id,series_id,series_description
0,44036939,2828203845,Sagittal T1
1,44036939,3481971518,Axial T2
2,44036939,3844393089,Sagittal T2/STIR


In [22]:
study_ids = list(df['study_id'].unique())

In [23]:
sample_sub = pd.read_csv(f'{INPUT_PATH}/sample_submission.csv')

In [24]:
LABELS = list(sample_sub.columns[1:])
LABELS

['normal_mild', 'moderate', 'severe']

In [25]:
CONDITIONS = [
    'spinal_canal_stenosis', 
    'left_neural_foraminal_narrowing', 
    'right_neural_foraminal_narrowing',
    'left_subarticular_stenosis',
    'right_subarticular_stenosis'
]

LEVELS = [
    'l1_l2',
    'l2_l3',
    'l3_l4',
    'l4_l5',
    'l5_s1',
]

In [26]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [44]:
import numpy as np
import cv2
import pydicom
import glob
from torch.utils.data import Dataset

class RSNA24TestDataset(Dataset):
    def __init__(self, df, study_ids, phase='test', transform=None):
        self.df = df
        self.study_ids = study_ids
        self.transform = transform
        self.phase = phase
    
    def __len__(self):
        return len(self.study_ids)
    
    def get_img_paths(self, study_id, series_desc):
        pdf = self.df[(self.df['study_id']==study_id) & (self.df['series_description']==series_desc)]
        
        # Find the series with the most images by actually counting files
        max_images = 0
        series_with_most_images = None
        for _, row in pdf.iterrows():
            series_path = f'{INPUT_PATH}/test_images/{study_id}/{row["series_id"]}'
            image_count = len([f for f in os.listdir(series_path) if f.endswith('.dcm')])
            if image_count > max_images:
                max_images = image_count
                series_with_most_images = row["series_id"]
        
        if series_with_most_images is None:
            return []
        
        allimgs = glob.glob(f'{INPUT_PATH}/test_images/{study_id}/{series_with_most_images}/*.dcm')
        return sorted(allimgs, key=natural_keys)
    
    def read_dcm_ret_arr(self, src_path):
        dicom_data = pydicom.dcmread(src_path)
        image = dicom_data.pixel_array
        image = (image - image.min()) / (image.max() - image.min() + 1e-6) * 255
        img = cv2.resize(image, (IMG_SIZE[0], IMG_SIZE[1]), interpolation=cv2.INTER_CUBIC)
        assert img.shape == (IMG_SIZE[0], IMG_SIZE[1])
        return img
    
    def select_images(self, allimgs, num_images, image_type):
        if len(allimgs) < num_images:
            print(f"Warning: Only {len(allimgs)} images available for {image_type}. Duplicating images to reach {num_images}.")
            # Duplicate images
            duplicated = []
            for i in range(num_images):
                duplicated.append(allimgs[i % len(allimgs)])
            return duplicated
        elif len(allimgs) == num_images:
            return allimgs
        else:
            if num_images == 10:  # For Sagittal T1 and Sagittal T2/STIR
                start = max(0, len(allimgs) // 2 - 5)
                return allimgs[start:start+10]
            elif num_images == 20:  # For Axial T2
                indices = np.linspace(0, len(allimgs) - 1, num_images, dtype=int)
                return [allimgs[i] for i in indices]
    
    def __getitem__(self, idx):
        x = np.zeros((IMG_SIZE[0], IMG_SIZE[1], IN_CHANS), dtype=np.uint8)
        st_id = self.study_ids[idx]
        
        image_types = [('Sagittal T1', 10), ('Sagittal T2/STIR', 10), ('Axial T2', 20)]
        
        for i, (series_desc, num_images) in enumerate(image_types):
            allimgs = self.get_img_paths(st_id, series_desc)
            if not allimgs:
                print(f"{st_id}: {series_desc}, has no images")
                continue
            
            selected_imgs = self.select_images(allimgs, num_images, series_desc)
            
            for j, img_path in enumerate(selected_imgs):
                try:
                    img = self.read_dcm_ret_arr(img_path)
                    x[..., j + i*10] = img.astype(np.uint8)
                except:
                    print(f'failed to load on {st_id}, {series_desc}')
        
        if self.transform is not None:
            x = self.transform(image=x)['image']
        x = x.transpose(2, 0, 1)
        
        return x, str(st_id)

In [45]:
transforms_test = A.Compose([
    A.Resize(IMG_SIZE[0], IMG_SIZE[1]),
    A.Normalize(mean=0.5, std=0.5)
])

In [46]:
test_ds = RSNA24TestDataset(df, study_ids, transform=transforms_test)
test_dl = DataLoader(
    test_ds, 
    batch_size=1, 
    shuffle=False,
    num_workers=N_WORKERS,
    pin_memory=True,
    drop_last=False
)

In [42]:
class RSNA24Model(nn.Module):
    def __init__(self, model_name, in_c=40, n_classes=75, pretrained=True, features_only=False):
        super().__init__()
        self.model = timm.create_model(
                                    model_name,
                                    pretrained=pretrained, 
                                    features_only=features_only,
                                    in_chans=in_c,
                                    num_classes=n_classes,
                                    global_pool='avg'
                                    )
    
    def forward(self, x):
        y = self.model(x)
        return y

In [31]:
import glob

models = []

CKPT_PATHS = glob.glob('/kaggle/input/rsna24lsdc-0816-train/rsna-2024-lsdc-trained-models/best_wll_model_fold-*.pt')
CKPT_PATHS = sorted(CKPT_PATHS)

for i, cp in enumerate(CKPT_PATHS):
    print(f'loading {cp}...')
    model = RSNA24Model(MODEL_NAME, IN_CHANS, N_CLASSES, pretrained=False)
    model.load_state_dict(torch.load(cp))
    model.eval()
    model.half()
    model.to(device)
    models.append(model)

loading /kaggle/input/rsna24lsdc-0816-train/rsna-2024-lsdc-trained-models/best_wll_model_fold-0.pt...
loading /kaggle/input/rsna24lsdc-0816-train/rsna-2024-lsdc-trained-models/best_wll_model_fold-1.pt...
loading /kaggle/input/rsna24lsdc-0816-train/rsna-2024-lsdc-trained-models/best_wll_model_fold-2.pt...
loading /kaggle/input/rsna24lsdc-0816-train/rsna-2024-lsdc-trained-models/best_wll_model_fold-3.pt...
loading /kaggle/input/rsna24lsdc-0816-train/rsna-2024-lsdc-trained-models/best_wll_model_fold-4.pt...


In [47]:
autocast = torch.cuda.amp.autocast(enabled=USE_AMP, dtype=torch.half)
y_preds = []
row_names = []

with tqdm(test_dl, leave=True) as pbar:
    with torch.no_grad():
        for idx, (x, si) in enumerate(pbar):
            x = x.to(device)
            pred_per_study = np.zeros((25, 3))
            
            for cond in CONDITIONS:
                for level in LEVELS:
                    row_names.append(si[0] + '_' + cond + '_' + level)
            
            with autocast:
                for m in models:
                    y = m(x)[0]
                    for col in range(N_LABELS):
                        pred = y[col*3:col*3+3]
                        y_pred = pred.float().softmax(0).cpu().numpy()
                        pred_per_study[col] += y_pred / len(models)
                y_preds.append(pred_per_study)

y_preds = np.concatenate(y_preds, axis=0)

100%|██████████| 1/1 [00:01<00:00,  1.19s/it]


In [48]:
sub = pd.DataFrame()
sub['row_id'] = row_names
sub[LABELS] = y_preds
sub.head(25)

,row_id,normal_mild,moderate,severe
0,44036939_spinal_canal_stenosis_l1_l2,0.302798,0.425369,0.271833
1,44036939_spinal_canal_stenosis_l2_l3,0.097608,0.475440,0.426952
2,44036939_spinal_canal_stenosis_l3_l4,0.075566,0.438784,0.485650
3,44036939_spinal_canal_stenosis_l4_l5,0.232463,0.242717,0.524820
4,44036939_spinal_canal_stenosis_l5_s1,0.821698,0.112532,0.065771
5,44036939_left_neural_foraminal_narrowing_l1_l2,0.440731,0.521838,0.037431
6,44036939_left_neural_foraminal_narrowing_l2_l3,0.224478,0.676013,0.099509
7,44036939_left_neural_foraminal_narrowing_l3_l4,0.187678,0.537752,0.274571
8,44036939_left_neural_foraminal_narrowing_l4_l5,0.148899,0.503269,0.347832
9,44036939_left_neural_foraminal_narrowing_l5_s1,0.168814,0.364438,0.466748


In [49]:
sub.to_csv('submission.csv', index=False)
pd.read_csv('submission.csv').head()

,row_id,normal_mild,moderate,severe
0,44036939_spinal_canal_stenosis_l1_l2,0.302798,0.425369,0.271833
1,44036939_spinal_canal_stenosis_l2_l3,0.097608,0.475440,0.426952
2,44036939_spinal_canal_stenosis_l3_l4,0.075566,0.438784,0.485650
3,44036939_spinal_canal_stenosis_l4_l5,0.232463,0.242717,0.524820
4,44036939_spinal_canal_stenosis_l5_s1,0.821698,0.112532,0.065771
